- #### Transformaciones
    - ##### sort/orderBy
    - ##### dropDuplicates
    - ##### distinct
    - ##### groupBy
    - ##### union/unionAll
    - ##### unionByName

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("sesion_1") \
        .master("local[*]") \
        .getOrCreate()

In [2]:

base_path = "../../resources/data/parquet/"
contracts_df = spark.read.parquet(base_path + "contracts_tmp")

contracts_df.show()
contracts_df.printSchema()

+-------+-----------+------------+------------+-----------+------------+----+--------+--------------+------+
|cod_iuc|cod_titular|cod_producto|fec_alta_ini|actual_date|fec_alta_fin|diff| calidad|prioridad_alta|activo|
+-------+-----------+------------+------------+-----------+------------+----+--------+--------------+------+
|  30000|      00006|         100|  2012-05-01| 2024-02-22|  2012-05-09|   8|    baja|          null|  true|
|  30001|      00006|         200|  2014-05-01| 2024-02-22|  2014-05-06|   5|    baja|          null|  true|
|  30002|      00006|         300|  2006-02-01| 2024-02-22|  2006-02-02|   1|    baja|          null|  null|
|  30003|      00006|         150|  2012-05-01| 2024-02-22|  2012-05-04|   3|    baja|          null|  true|
|  30002|      00005|         300|  2012-05-01| 2024-02-22|  2012-05-08|   7|    baja|          null|  true|
|  30004|      00006|         400|  2012-05-01| 2024-02-22|  2012-05-04|   3|   media|          null|  null|
|  30005|      0000

In [3]:
# sort/orderBy - operación que invokucra shuffle

import pyspark.sql.functions as f

contracts_df.orderBy(f.col("cod_iuc").desc(), f.col("fec_alta_fin").desc()).show()

+-------+-----------+------------+------------+-----------+------------+----+--------+--------------+------+
|cod_iuc|cod_titular|cod_producto|fec_alta_ini|actual_date|fec_alta_fin|diff| calidad|prioridad_alta|activo|
+-------+-----------+------------+------------+-----------+------------+----+--------+--------------+------+
|  30011|      00003|        1100|  2018-10-01| 2024-02-22|  2018-10-04|   3|muy alta|            ok|  true|
|  30010|      00007|        1000|  2014-08-01| 2024-02-22|  2014-08-08|   7|    alta|          null|  null|
|  30010|      00006|        1000|  2014-02-01| 2024-02-22|  2014-02-06|   5|    alta|            ok|  true|
|  30009|      00006|         900|  2015-09-01| 2024-02-22|  2015-09-07|   6|    alta|            ok|  true|
|  30009|      00002|         900|  2009-10-01| 2024-02-22|  2009-10-05|   4|    alta|            ok|  true|
|  30008|      00002|         800|  2014-03-21| 2024-02-22|  2014-03-29|   8|    alta|            ok|  true|
|  30008|      0000

In [4]:
# dropDuplicates distinct

#contracts_df.dropDuplicates(["cod_iuc", "cod_producto"]).show()

#contracts_df.distinct().show() # - .distinct() Elimina valores duplicados

In [5]:
# groupBy -> count, max, min, avg, mean, sum

contracts_df.groupBy("cod_iuc").count().show()
#contracts_df.groupBy(f.col("cod_iuc")).sum("diff").show()
#contracts_df.groupBy(f.col("cod_iuc")).max("fec_alta_ini").show()

contracts_df.groupBy(f.col("cod_iuc"), f.col("activo"))\
    .agg(
        f.count("*").alias("count"),
        f.sum("diff").alias("sum_diff"),
        f.max("diff").alias("max_diff"),
        f.min("diff").alias("min_diff"),
        f.mean("diff").alias("mean_diff"),
        f.avg("diff").alias("avg_diff"),
        f.max("fec_alta_ini").alias("max_fec_alta_ini")
    ).show()


+-------+-----+
|cod_iuc|count|
+-------+-----+
|  30002|    3|
|  30005|    2|
|  30006|    2|
|  30009|    2|
|  30008|    3|
|  30007|    5|
|  30004|    2|
|  30003|    5|
|  30000|    2|
|  30011|    1|
|  30001|    3|
|  30010|    2|
+-------+-----+

+-------+------+-----+--------+--------+--------+-----------------+-----------------+----------------+
|cod_iuc|activo|count|sum_diff|max_diff|min_diff|        mean_diff|         avg_diff|max_fec_alta_ini|
+-------+------+-----+--------+--------+--------+-----------------+-----------------+----------------+
|  30002|  null|    2|       7|       6|       1|              3.5|              3.5|      2006-02-01|
|  30006|  null|    1|       6|       6|       6|              6.0|              6.0|      2012-05-01|
|  30001|  true|    3|      16|       9|       2|5.333333333333333|5.333333333333333|      2019-11-10|
|  30003|  true|    5|      23|       7|       2|              4.6|              4.6|      2020-03-01|
|  30002|  true|    1|

In [6]:
contracts_df\
    .groupBy("cod_iuc") \
    .pivot("activo") \
    .agg(
        f.count("*"),
        f.max("fec_alta_fin"),
        f.avg("diff")
    ) \
    .show()

+-------+-------------+----------------------+--------------+-------------+----------------------+-----------------+
|cod_iuc|null_count(1)|null_max(fec_alta_fin)|null_avg(diff)|true_count(1)|true_max(fec_alta_fin)|   true_avg(diff)|
+-------+-------------+----------------------+--------------+-------------+----------------------+-----------------+
|  30002|            2|            2006-02-02|           3.5|            1|            2012-05-08|              7.0|
|  30005|            0|                  null|          null|            2|            2012-05-03|              5.5|
|  30006|            1|            2012-05-07|           6.0|            1|            2016-12-10|              0.0|
|  30009|            0|                  null|          null|            2|            2015-09-07|              5.0|
|  30008|            0|                  null|          null|            3|            2014-03-29|6.333333333333333|
|  30007|            1|            2014-02-01|           0.0|   

In [7]:
## union/unionAll unionByName
contracts_tmp_df = contracts_df.filter(f.col("activo").isNull())

contracts_tmp_df.show()

contracts_tmp_df.select("cod_titular","fec_alta_ini","actual_date")\
    .union(contracts_tmp_df.select("cod_titular","fec_alta_ini","actual_date")).show()

contracts_tmp_df.select("cod_titular","fec_alta_ini","actual_date")\
    .unionByName(contracts_tmp_df.select("actual_date","cod_titular","fec_alta_ini")).show()

+-------+-----------+------------+------------+-----------+------------+----+-------+--------------+------+
|cod_iuc|cod_titular|cod_producto|fec_alta_ini|actual_date|fec_alta_fin|diff|calidad|prioridad_alta|activo|
+-------+-----------+------------+------------+-----------+------------+----+-------+--------------+------+
|  30002|      00006|         300|  2006-02-01| 2024-02-22|  2006-02-02|   1|   baja|          null|  null|
|  30004|      00006|         400|  2012-05-01| 2024-02-22|  2012-05-04|   3|  media|          null|  null|
|  30006|      00006|         600|  2012-05-01| 2024-02-22|  2012-05-07|   6|  media|          null|  null|
|  30007|      00006|         700|  2014-02-01| 2024-02-22|  2014-02-01|   0|   alta|          null|  null|
|  30002|      00007|         300|  2001-04-11| 2024-02-22|  2001-04-17|   6|   baja|          null|  null|
|  30004|      00002|         400|  2008-02-01| 2024-02-22|  2008-02-10|   9|  media|          null|  null|
|  30010|      00007|       